In [1]:
!conda info || true
!python --version
# install and add to requirements.txt
!touch requirements.txt
!echo -n > requirements.txt

# add dependency in requirements.txt
!echo typing >> requirements.txt
!echo pydantic >> requirements.txt
!echo sentence-transformers >> requirements.txt # https://sbert.net/docs/quickstart.html
!echo langchain[all] >> requirements.txt # https://langchain.readthedocs.io/en/latest/getting_started/getting_started.html

!time pip install -r requirements.txt
!pip freeze > requirements.frozen.tmp.txt
!cat requirements.frozen.tmp.txt | grep -v "file://" > requirements.frozen.txt 
!rm -f requirements.frozen.tmp.txt


     active environment : None
       user config file : /home/jovyan/.condarc
 populated config files : /opt/conda/.condarc
          conda version : 22.11.1
    conda-build version : not installed
         python version : 3.10.8.final.0
       virtual packages : __archspec=1=x86_64
                          __glibc=2.35=0
                          __linux=5.15.49=0
                          __unix=0=0
       base environment : /opt/conda  (writable)
      conda av data dir : /opt/conda/etc/conda
  conda av metadata url : None
           channel URLs : https://conda.anaconda.org/conda-forge/linux-64
                          https://conda.anaconda.org/conda-forge/noarch
          package cache : /opt/conda/pkgs
                          /home/jovyan/.conda/pkgs
       envs directories : /opt/conda/envs
                          /home/jovyan/.conda/envs
               platform : linux-64
             user-agent : conda/22.11.1 requests/2.28.2 CPython/3.10.8 Linux/5.15.49-linuxkit ubu

In [2]:
# create directory structure
!mkdir -p data_processing
!touch data_processing/__init__.py

In [3]:
%%writefile data_processing/data_processing.py


def hello():
    pass

Writing data_processing/data_processing.py


In [4]:
from data_processing.data_processing import hello

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
sentence_embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [-1.37173468e-02 -4.28515449e-02 -1.56285837e-02  1.40537750e-02
  3.95538211e-02  1.21796295e-01  2.94333491e-02 -3.17524038e-02
  3.54959629e-02 -7.93140009e-02  1.75878983e-02 -4.04369719e-02
  4.97259349e-02  2.54912004e-02 -7.18701035e-02  8.14968795e-02
  1.47068314e-03  4.79626879e-02 -4.50336263e-02 -9.92174745e-02
 -2.81769708e-02  6.45046160e-02  4.44670804e-02 -4.76217009e-02
 -3.52952406e-02  4.38671410e-02 -5.28566092e-02  4.33077395e-04
  1.01921506e-01  1.64072327e-02  3.26996520e-02 -3.45986784e-02
  1.21339010e-02  7.94870853e-02  4.58346074e-03  1.57778412e-02
 -9.68204252e-03  2.87625883e-02 -5.05805947e-02 -1.55793512e-02
 -2.87906155e-02 -9.62282624e-03  3.15556899e-02  2.27348879e-02
  8.71449113e-02 -3.85027081e-02 -8.84718448e-02 -8.75496585e-03
 -2.12343428e-02  2.08923165e-02 -9.02077779e-02 -5.25732227e-02
 -1.05638811e-02  2.88310610e-02 -1.61455069e-02  6.17837021e-03
 -1.23234

In [5]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

#Sentences are encoded by calling model.encode()
emb1 = model.encode("This is a red cat with a hat.")
emb2 = model.encode("Have you seen my red cat?")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.6153]])
